In [15]:
system("sudo apt-get update -y && sudo apt-get install -y jags && sudo apt-get install -y r-cran-rjags")
install.packages('runjags')
install.packages('rjags')

In [16]:
# Charger les bibliothèques nécessaires
library(dplyr)
library(ggplot2)
library(tidyverse)
library(runjags)
library(rjags)

# Introduction
L’étude des disparités salariales est un sujet central en économie du travail et en sciences sociales. Malgré les avancées vers l’égalité des chances, des différences de salaires persistent entre divers groupes sociaux, notamment en fonction du genre, de l’ethnicité, du niveau d’éducation et de l’expérience professionnelle. Aux États-Unis, des études ont montré que les écarts salariaux sont significatifs : les femmes gagnent en moyenne moins que les hommes, et certaines minorités ethniques, comme les Afro-Américains et les Latino-Américains, perçoivent des salaires inférieurs à ceux des Américains blancs et asiatiques.

Notre projet vise à identifier les principaux facteurs influençant les salaires horaires en utilisant des modèles bayésiens. Plus précisément, nous cherchons à quantifier l’impact de variables telles que le genre, l’expérience professionnelle, le niveau d’éducation, l’appartenance syndicale et la localisation géographique sur les salaires.

Pour ce faire, nous utiliserons des méthodes bayésiennes de régression mises en œuvre avec JAGS, permettant une estimation robuste et une meilleure quantification des incertitudes. Notre approche inclura l’exploration des données, la spécification des modèles avec des choix de lois a priori justifiées, ainsi que des évaluations rigoureuses des performances des modèles à travers des diagnostics de convergence et des simulations de données fictives (fake data check).

Ce rapport est structuré comme suit : après une présentation des données et de leur exploration, nous décrirons les modèles bayésiens retenus et justifierons les choix de lois a priori. Nous présenterons ensuite les résultats de l’ajustement des modèles, suivis des diagnostics de convergence et de l’évaluation de leur performance. Enfin, nous discuterons des conclusions et des implications de nos résultats dans le contexte des disparités salariales aux États-Unis.

# Modèle génératif et espace des paramètres

On veut construire un modèle prédictif pour le salaire des individus pour lesquels on connait 10 variables explicatives (âge, éducation, experience professionnelle, etc.), 434 individus dans un jeu de données d'entraînement et 100 individus dans le jeu de données de test.

On note $Y_i \in \mathbb{R}$ le salaire de l'individu $i$ et $X_i \in \mathbb{R}^{10}$ les variables explicatives de l'individu $i$.

Régression linéaire:
$Y_i = \beta_0 + \beta_1 X_{1i} + \beta_2 X_{2i} + \cdots + \beta_{10} X_{10i} + \sigma \epsilon_i$
11 paramètres $\beta = (\beta_0, \beta_1, \beta_2, \cdots, \beta_{10})$  $\epsilon_i \sim \mathcal{N}(0, 1)$ et $\sigma \in \mathbb{R^+}$

Certaines variables peuvent avoir peu ou pas d'effet sur le salaire, ie des $\beta_i = 0$.

## Distribution à postériori

A partir de $y=(y_1, y_2, \cdots, y_{434})$ et $ X=(X_1, X_2, \cdots, X_{434})$, on peut calculer $P(\beta | y, X)$ et regarder $P(\beta_j \neq 0 | y, X)$ pour chaque coefficient.

## Performances prédictives

Un manière d'évaluer la performance prédictive $\hat{\beta}_{bayes} = E(\beta | y, X)$
$X_{test}$ matrice 100x10
$y_{test}$ les observations
$y_{test,Bayes} = X_{test} \hat{\beta}_{bayes}$

# Degrès de croyance

Que croit-on sur l'effet de chaque variable explicative sur le salaire?

Nous pouvons formuler des hypothèse à priori sur l'impact des différentes variables :

- Éducation : On s’attend à une relation positive entre le niveau d’éducation et le salaire.

- Expérience professionnelle (workexp) : Plus d’expérience est généralement associée à un salaire plus élevé.

- Sexe (female) : Il est bien documenté qu’un écart salarial entre hommes et femmes existe (gender pay gap).

- Région (south) : Il peut y avoir des différences salariales régionales.

- Appartenance syndicale (unionmember) : Être syndiqué peut influencer le salaire.

- Âge (age) : En général, l’âge est corrélé à l’expérience professionnelle, mais il pourrait aussi capturer d’autres dynamiques comme la discrimination liée à l’âge.

- Ethnicité (ethnicity) : Certaines inégalités salariales peuvent exister selon l’origine ethnique.

- Occupation et secteur (occupation, sector) : Le type d’emploi et le secteur d’activité influencent fortement le salaire.

- Statut matrimonial (married) : Certaines études suggèrent que les personnes mariées ont des salaires plus élevés, en particulier les hommes.


### Estimation d'un paramètre $\beta$ avec des données

Ayant obtenu {$Y=y$} , $P(\beta | y,X) = \frac{P(\beta) P(y,X | \beta)}{P(y,X)}$

Donc $posterior \propto prior \times likelihood$

$prior = P(\beta)$ : Croyance des effets des variables sur le salaire avant d'avoir les données.
$likelihood = P(y | \beta)$ : Probabilité d'observer les données en fonction des paramètres du modèle
$posterior = P(\beta | y)$ : La Distribution à posteriori après avoir observé les données

On peut le voir ainsi :

$P(Model | New Data)  = \frac{P(Model) P(New Data | Model)}{P(New Data)}$



# Nos données

Notre jeu de données comprend 534 individus. Chaque individu a 10 variables : education, south, female, workexp, unionmember, wages, age, ethnicity, occupation, sector, married.

- Variables quantitatives:
    > **education**: Années d'études
    >**workexp**: Années d'expêrience professionnelle
    >**age**: âge

- Variables qualitatives:
    >**south**: Vit dans le sud ou non
    >**female**: Indicateur de genre (0 = Homme, 1 = Femme)
    >**unionmember**: Indicateur de membre du syndicat (0 = Non, 1 = Oui)
    >**married**: Indicateur de statut marital. Marié(e) ou non.

- Variables categorielles:
    >**ethnicity**: Ethnicité (White, Hispanic, Other)
    >**occupation**: Occupation (Management, Sales, Clerical, Professional, Service, Other)
    >**sector**: Secteur d'activité (Manufactoring, Construction, Other)




On ajoute une colonne pour calculer la moyenne de la variable wage avec 0 pour les salaires inférieurs à la moyenne et 1 pour les salaires supérieurs à la moyenne.

In [3]:
# Charger les données
wage_data <- read.csv("https://raw.githubusercontent.com/derghalmanal/Wage/refs/heads/main/wage_data.csv", header=TRUE, sep=",")

# Calculer la moyenne de la colonne wage
mean_wage <- mean(wage_data$wage, na.rm=TRUE)

# Ajouter la colonne wage_moyenne
wage_data$wage_level <- ifelse(wage_data$wage < mean_wage, 0, 1)

# Transformer la colonne wage_moyenne en facteur
wage_data$wage_level <- as.factor(wage_data$wage_level)

# Vérifier le résultat
head(wage_data)

In [4]:
str(wage_data)

Analysons la contribution de chaque variable aux salaires

Dans toute la suite du rapport l = lower pour signifier que le salaire est inférieur à la moyenne et h = higher pour signifier que le salaire est supérieur à la moyenne.

On cherchera la loi de la vraisemblance $X_{i,s}|\beta$ pour chaque individu $i$ et chaque variable $s \in \{l,h\}$.

In [5]:
plot_credible_intervals <- function(fit) {
  # Extract the MCMC samples and the names of the parameters
  samples <- as.data.frame(jags_samples$mcmc %>% lapply(as_tibble) %>% bind_rows())
  params <- names(samples)

  # Calculate the 50% and 95% credible intervals for each parameter
  intervals <- data.frame(param = character(), lower = numeric(), upper = numeric())
  for (param in params) {
    est_mean <- mean(samples[[param]])
    est_median <- median(samples[[param]])
    ci_50_infCI <- quantile(samples[[param]], probs = 0.25)
    ci_50_supCI <- quantile(samples[[param]], probs = 0.75)
    ci_95_infCI <- quantile(samples[[param]], probs = 0.025)
    ci_95_supCI <- quantile(samples[[param]], probs = 0.975)
    intervals <- rbind(intervals, data.frame(param = param, est_mean = est_mean, est_median = est_median, ci_50_infCI = ci_50_infCI, ci_50_supCI = ci_50_supCI, ci_95_infCI = ci_95_infCI, ci_95_supCI = ci_95_supCI))
  }

  # Create a ggplot object
  p <- ggplot(intervals, aes(y = param)) +
    theme_classic() +
    geom_segment(aes(y = param, yend = param, x = ci_95_infCI, xend = ci_95_supCI),
      color = "red", size = 0.5
    ) +
    geom_segment(aes(y = param, yend = param, x = ci_50_infCI, xend = ci_50_supCI),
      color = "red", size = 1.5
    ) +
    geom_point(aes(x = est_mean), size = 3) +
    labs(title = "Posterior credible intervals") +
    xlab("") +
    ylab("")

  # Print the plot
  print(p)
}

# Histogramme des variables

In [6]:
numeric_vars <- c("education", "workexp")
wage_data[numeric_vars] <- lapply(wage_data[numeric_vars], function(x) as.numeric(as.character(x)))

for (var in numeric_vars){
    print(ggplot(data = wage_data, aes_string(x = var, fill = "wage_level" )) +
      geom_bar( position = "identity", alpha = 0.7) +
      labs(title = paste0("Histogramme de la variable ", var , " en fonction du salaire moyen"),
           x = var, y = "Fréquence") +
      scale_fill_manual(values = c("blue", "red")) +  # Couleurs des labels
      theme_minimal())
}

wage_data$south        <- factor(wage_data$south, labels = c("Non-South", "South"))
wage_data$female       <- factor(wage_data$female, labels = c("Male", "Female"))
wage_data$unionmember  <- factor(wage_data$unionmember, labels = c("Non-Member", "Member"))
wage_data$married      <- factor(wage_data$married, labels = c("Single", "Married"))

# Now binary_vars are factors with clear labels
binary_vars <- c("south", "female", "unionmember", "married")

# Plot
for (var in binary_vars){
  print(
    ggplot(data = wage_data, aes_string(x = var, fill = "wage_level")) +
      geom_bar(position = "dodge", alpha = 0.7) +
      labs(title = paste("Histogramme de la variable", var, "en fonction du niveau de salaire"),
           x = var, y = "Fréquence") +
      scale_fill_manual(values = c("blue", "red"), 
                        labels = c("Salaire < Moyenne", "Salaire ≥ Moyenne")) +
      theme_minimal()
  )
}
categorial_vars <- c("ethnicity", "sector","occupation")
wage_data[categorial_vars] <- lapply(wage_data[categorial_vars], function(x) as.factor(as.character(x)))

for (var in categorial_vars){
    print(ggplot(data = wage_data, aes_string(x = var, fill = "wage_level" )) +
      geom_bar( position = "identity", alpha = 0.7) +
      labs(title = paste0("Histogramme de la variable ", var , " en fonction du salaire moyen"),
           x = var, y = "Fréquence") +
      scale_fill_manual(values = c("blue", "red")) +  # Couleurs des labels
      theme_minimal())
}

Choix de la vraisemblance :

Une distribution normale semble appropriée pour modéliser les années d'expérience au travail, car les données suivent une forme de cloche.

$y_{s},X | \beta_{workexp,s} \sim \mathcal{Normal}(\beta_{workexp,s})$

Avec $\beta_{workexp,s} = (\mu_s, \sigma_s)$ et $ s \in \{l,h\}$


# Variable South

<img src="image-20250330-164413.png" width="" align="" />

## Vraisemblance

Étant donné que "south" est une variable binaire (Non-South, South), une distribution de Bernoulli est appropriée pour modéliser cette variable. La fonction de vraisemblance pour une variable de Bernoulli est donnée par : 
$$
L(\theta \mid x) = \theta^x (1 - \theta)^{1 - x}
$$

## Prior approprié

Pour le prior, une distribution Beta est souvent utilisée comme conjugate prior pour une distribution de Bernoulli. La distribution Beta est définie par :

$$
P(\theta_{\text{s}} \mid \alpha_{\text{s}} , \beta_{\text{s}}) = \frac{\theta_{\text{s}}^{\alpha_{\text{s}} - 1} (1 - \theta_{\text{s}})^{\beta_{\text{s}} - 1}}{B(\alpha_{\text{s}}, \beta_{\text{s}})}
$$
ou  $$ B(\alpha, \beta) $$ est la fonction Beta de normalisation, et $$\alpha_{\text{s}}  et  \beta_{\text{s}}$$ sont les paramètres de forme de la distribution avec 
$$\alpha_{\text{s}} = \beta_{\text{s}} = 1 pour s \in \{l, h\}$$

Choix des paramètres du prior :  


## Implémentation en Jags

## MCMC 

## Fake data check

## Conclusion

# Variable Education

La variable Education représente le nombre d'années d'études.

<img src="image-20250330-163929.png" width="" align="" />

## Vraisemblance

Choix de la vraisemblance :

Étant donné la concentration des données à une valeur spécifique, nous avons choisi une distribution de Poisson pour modéliser le nombre d'années d'études. 

$y_{s},X | \beta_{education,s} \sim \mathcal{Poisson}(\beta_{education,s} )$ et $s \in \{l,h\}$

## Prior

L'histogramme montre une distribution avec un pic très prononcé à 12 années d'études, indiquant que la plupart des individus ont ce niveau d'éducation.
Il y a quelques valeurs dispersées autour de ce pic, mais la majorité des données est concentrée à 12 ans.


$\beta_{education} \sim \mathcal{Gamma}(\alpha, \beta)$

On souhaite que $E(\beta_{education,l}) = 12$ et $V(\beta_{education,l}) = 20$

On souhaite que $E(\beta_{education,s}) = 12$ et $V(\beta_{education,s}) = 20$

Pour la loi gamma $E(\beta) = \frac{\alpha}{\beta}$ et $V(\beta) = \frac{\alpha}{\beta^2}$

donc $\beta_{education} \sim \mathcal{Gamma}(7.2, 0.6)$

## Implementation en JAGS

In [17]:
model_string_education <- "
model {

  # Priors
  beta_education_l~ dgamma(7.2, 0.6)
  beta_education_h ~ dgamma(7.2, 0.6)
  
  
  for (i in 1:n1) {
    y1[i] ~ dpois(beta_education_l)
  }
  
  for (i in 1:n2) {
    y2[i] ~ dpois(beta_education_h)
  }
}
"
education_l <- wage_data$education[wage_data$wage_level == 0]
education_h <- wage_data$education[wage_data$wage_level == 1]

# Définition des données
y1 <- education_l
y2 <- education_h
data_list <- list(y1 = y1, y2 = y2, n1 = length(y1), n2 = length(y2))


# Compiling and producing posterior samples from the model.
jags_samples_education <- run.jags(model = model_string_education, data = data_list, monitor = c("beta_education_l", "beta_education_h"), n.chains = 3, adapt = 1000, burnin = 1000, thin = 1)

# Plotting and summarizing the posterior distribution
jags_samples_education
plot_credible_intervals(jags_samples_education)

## MCMC

In [30]:
# Fonction pour tracer traceplots et autocorrélogrammes
plot_mcmc <- function(jags_samples) {
  
  # Convertir en objet mcmc
  mcmc_samples <- as.mcmc.list(jags_samples)
  
  # Traceplots (convergence)
  plot(mcmc_samples, trace = TRUE, density = TRUE)
  
  # Tracer l'autocorrélation pour chaque chaîne
  autocorr.plot(jags_samples)
}

# Utilisation de la fonction
plot_mcmc(jags_samples_education)



## Fake data check

In [31]:
generate_fake_data <- function(n1, n2) {
  beta_education_l_fake <- 12  
  beta_education_h_fake <- 13
  
  # Générer des données factices à partir de la distribution de Poisson
  y1_fake <- rpois(n1, beta_education_l_fake)
  y2_fake <- rpois(n2, beta_education_h_fake)
  
  return(list(y1_fake = y1_fake, y2_fake = y2_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(education_l), length(education_h))

# Comparer les données factices avec les données réelles
summary(fake_data$y1_fake)
summary(fake_data$y2_fake)
summary(y1)
summary(y2)

data_list <- list(y1 = fake_data$y1_fake, y2 = fake_data$y2_fake, n1 = length(fake_data$y1_fake), n2 = length(fake_data$y2_fake))


# Compiling and producing posterior samples from the model.
jags_samples_education <- run.jags(model = model_string_education, data = data_list, monitor = c("beta_education_l", "beta_education_h"))

# Plotting and summarizing the posterior distribution
jags_samples_education
plot_credible_intervals(jags_samples_education)
summary(jags_samples_education)

## Conclusion

# Variable Age

On observe que l'age des individus est fortement correle avec l'experience professionnelle, ce qui fait sens puisque plus l'age est grand, plus l'experience professionnelle est longue. Pour éviter une instabilité des estimations, nous supprimons écartons la variable Age.

In [15]:
ggplot(wage_data, aes(x=age, y=workexp)) +
  geom_point(alpha=0.5, color="purple") +
  labs(title="Relation entre Age et Expérience Professionnelle", x="Age", y="Expérieence Professionnelle")

# Variable Work experience

<img src="image-20250330-163826.png" width="" align="" />

## Vraisemblance

Une distribution normale semble appropriée pour modéliser les années d'expérience au travail, car les données suivent une forme de cloche.

$y_{s},X | \beta_{workexp,s} \sim \mathcal{Normal}(\beta_{workexp,s})$

Avec $\beta_{workexp,s} = (\mu_s, \sigma_s)$ et $ s \in \{l,h\}$

## Choix du prior

$\mu_s \sim \mathcal{Uniform}(0, 30)$ 
$\sigma_s \sim \mathcal{Uniform}(0, 50)$

## Implémentation en JAGS

In [14]:
model_string_workexp <- "
model {

  mu_l ~ dunif(0, 30)
  sigma_l ~ dunif(0, 50)
  precision_l <- 1/sigma_l^2

  mu_h ~ dunif(0, 30)
  sigma_h ~ dunif(0, 50)
  precision_h <- 1/sigma_h^2
  
  for (i in 1:n1) {
    y1[i] ~ dnorm(mu_l, precision_l)
  }
  
  for (i in 1:n2) {
    y2[i] ~ dnorm(mu_h, precision_h)
  }

  
}
"

# Définition des données
y1 <- wage_data$workexp[wage_data$wage_level == 0]
y2 <- wage_data$workexp[wage_data$wage_level == 1]
data_list <- list(y1 = y1, y2 = y2, n1 = length(y1), n2 = length(y2))



# Compiling and producing posterior samples from the model.
jags_samples_workexp <- run.jags(model = model_string_workexp, data = data_list, monitor = c("mu_l", "sigma_l", "mu_h", "sigma_h"), n.chains = 3, adapt = 1000, burnin = 1000, thin = 1)

# Plotting and summarizing the posterior distribution
jags_samples_workexp
plot_credible_intervals(jags_samples_workexp)

## MCMC

In [33]:
plot_mcmc(jags_samples_workexp)

## Fake data check

In [13]:
generate_fake_data <- function(n1, n2) {
    mu_l_fake <- 15
    sigma_l_fake <- 12
    mu_h_fake <- 17
    sigma_h_fake <- 10
  
  # Générer des données factices à partir de la distribution de Poisson
  y1_fake <- rnorm(n1, mu_l_fake, sigma_l_fake)
  y2_fake <- rnorm(n2, mu_h_fake, sigma_h_fake)
  
  return(list(y1_fake = y1_fake, y2_fake = y2_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(y1), length(y2))

# Comparer les données factices avec les données réelles
summary(fake_data$y1_fake)
summary(fake_data$y2_fake)
summary(y1)
summary(y2)

data_list <- list(y1 = fake_data$y1_fake, y2 = fake_data$y2_fake, n1 = length(fake_data$y1_fake), n2 = length(fake_data$y2_fake))


# Compiling and producing posterior samples from the model.
jags_samples_workexp <- run.jags(model = model_string_workexp, data = data_list, monitor = c("mu_l", "sigma_l", "mu_h", "sigma_h"))

# Plotting and summarizing the posterior distribution
jags_samples_workexp
plot_credible_intervals(jags_samples_workexp)
summary(jags_samples_workexp)

# Variable female

<img src="image-20250330-165243.png" width="" align="" />

## Vraisemblance

Chaque observation suit une loi de Bernoulli :

$$
y_i \sim \text{Bernoulli}(p_i)
$$
avec : 
$$
\text{logit}(p_i) = \log\left(\frac{p_i}{1 - p_i}\right) = \beta_0 + \beta_1 \cdot \text{female}_i
$$
- $y_i = 1$ si le salaire est ≥ à la moyenne, 0 sinon  
- $\text{female}_i = 1$ si femme, 0 si homme

## Choix du Prior

- **Effet de base pour un homme ($\beta_0$)** : 

$$
\beta_0 \sim \mathcal{N}(0, 2.5^2)
$$

- **Effet d’être une femme ($\beta_1$)** :

$$
\beta_1 \sim \mathcal{N}(-0.5, 1^2)
$$

## Implémentation en JAGS

In [12]:
# ------------------------------
# 1. Définir le modèle JAGS
# ------------------------------
model_string_female <- "
model {

  # Priors
  beta0 ~ dnorm(0, 0.16)        # équivaut à N(0, 2.5^2)
  beta1 ~ dnorm(-0.5, 1.0)      # équivaut à N(-0.5, 1^2)

  # Vraisemblance
  for (i in 1:N) {
    logit(p[i]) <- beta0 + beta1 * female[i]
    y[i] ~ dbern(p[i])
  }

}
"

# ------------------------------
# 2. Préparer les données
# ------------------------------
# Cible binaire : 0 ou 1
y <- as.numeric(as.character(wage_data$wage_level))  # s'assurer que c'est bien numérique (0/1)
female <- as.numeric(wage_data$female)               # 0 = homme, 1 = femme

data_list_female <- list(
  y = y,
  female = female,
  N = length(y)
)

# ------------------------------
# 3. Exécuter le modèle avec run.jags
# ------------------------------
library(runjags)

jags_samples_female <- run.jags(
  model = model_string_female,
  data = data_list_female,
  monitor = c("beta0", "beta1"),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000,
  thin = 1
)

# ------------------------------
# 4. Résultats
# ------------------------------
jags_samples_female
plot_credible_intervals(jags_samples_female)


## MCMC

In [11]:
plot_mcmc(jags_samples_female)

## Fake data check

## Conclusion

# Variable Married

<img src="image-20250330-164951.png" width="" align="" />

## Vraisemblance


Etant donné que "married" est une variable binaire (Single, Married), une distribution de Bernoulli est appropriée pour modéliser cette variable. La fonction de vraisemblance pour une variable de Bernoulli est donnée par :
$$
L(\theta \mid x) = \theta^x (1 - \theta)^{1 - x}
$$
où $$x=1$$ si la personne est mariée, $$0$$ sinon.
et $$\theta$$ est la probabilité d'être marié.


## Choix du Prior

En ce qui concerne le choix du prior, la distribution Beta est couramment utilisée comme prior conjugué pour une distribution de Bernoulli. Elle est définie par la fonction de normalisation $$ B(\alpha, \beta) $$ avec $$\alpha $$ et $$\beta $$ les paramètres de forme de la distribution.  
$$
P(\theta \mid \alpha, \beta) = \frac{\theta^{\alpha - 1} (1 - \theta)^{\beta - 1}}{B(\alpha, \beta)}
$$
Pour le choix des paramètres du prior, on considère qu'il n'est pas informatif donc $$\alpha=1$$, $$\beta=1$$.


## Implémentation en JAGS

## MCMC

## Fake data check

## Conclusion

# Variable unionmember

<img src="image-20250330-165832.png" width="" align="" />

## Vraisemblance

## Choix du prior

## Implémentation en JAGS

## MCMC

## Fake data check

## Conclusion

# Variable ethnicity

<img src="image-20250330-170259.png" width="" align="" />

## Vraisemblance

## Choix du prior

## Implémentation en JAGS

## MCMC

## Fake data check

## Conclusion

# Variable sector

<img src="image-20250330-170309.png" width="" align="" />

## Vraisemblance

## Choix du prior

## Implémentation en JAGS

## MCMC

## Fake data check

## Conclusion

# Variable occupation

<img src="image-20250330-170318.png" width="" align="" />

## Vraisemblance

## Choix du prior

## Implémentation en JAGS

## MCMC

## Fake data check

## Conclusion

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=650b0311-43bb-4704-b320-0c62dbd2dedc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>